<p style="text-align: center;font-size: 40pt">Use case 3</p>

\section{Autonomous Driving}\label{sec:autoDriving}
%=========================================================

% Describe the application
The recent expansion of the autonomous driving field have been pushed forwards by car companies that teamed up with research partners, like Volkswagen in the EU project V-Charge, or by large company like Google hiring roboticists to develop new car prototypes.
The American state of Nevada even officially delivered its first license for driverless car in May 2012.
Although photometric registration is more attractive from the industry point of view, due to the potential low cost of cameras, geometric dense maps can support other activities.
For example, road inspection is a tedious task that is mainly realized visually by operators driving on the roads.
Large-scale road construction sites can also profit from a fast monitoring of the work progress.
Applications usually target urban environments, which are mostly structured (e.g., road, buildings) or semi-structured when the vegetation is more prominent.
The environment is predominantly static, but a large part of the field of view can be occupied by other cars such as in dense traffic situations.

% Describe the platform
The SmartTer (\autoref{fig:robotSmartter}) was a modified version of a Smart Fortwo.
The car was developed by the \acx{asl} and served in 2006 has a technological demonstrator in the European SPARC project realized in collaboration with Daimler Chrysler.
The Smart is one of the most compact car with a volume of 6.38\,m$^3$ and a weight of 730\,kg.
Two SICK LMS-291 laser rangefinders were mounted on a vertical rotating axis, each of them providing 14,000 points every second.
Motion compensations were applied to the 3D scans to cope with the high speed of the vehicle (15\,km/h).
Other sensors included navigation laser, omnidirectional camera, monocular camera, \acx{gps} and \acx{imu}.
The overall motion of the vehicle is expected to be smooth, with a strong assumption of translation on the xy-plane.
For a deeper description of the vehicle, we refer the reader to the publication of \citet{Lamon:2006tc}.

\begin{figure}[htb]
\centering
	\includegraphics[width=0.65\textwidth]{robot_smartter}
\caption{The autonomous car, named SmartTer, with its suite of five SICK LMS-291 laser rangefinders.
Larger wheels were installed to allow off-road driving.
}
\label{fig:robotSmartter}
\end{figure}

% Describe the solution
The solution selected for this application had to deal with a large scanned volume and noises caused by the velocity of the vehicle.
We drastically reduced the number of inputted points by keeping a maximal density of 0.5 points per m$^3$.
For the registration, we cut points beyond the maximal reach of the sensor to reduce the \acx{nn} searching space.
The pre-alignment of the scans was fairly accurate, but we kept a maximum matching distance of 1.5\,m to recover from loop-closing error.
Many large trees along the road were having their surface normal wrongly estimated due to their unstructured nature, which increased the point-to-plane alignment error.
With that in mind, we used a point-to-point error metric for the minimization, as opposed to other solutions presented in this section.
A denser global representation was maintained to ensure for more stability of the registration, especially on the ground where the density of a single scan was dropping rapidly.


\begin{table}[htb]
\caption{Configuration of the \icp chain for the SmartTer mapping applications.
The definition of the column \emph{Step} follows \autoref{chap:icpSolutions}. 
The names used in the column \emph{Module} refer to specific implementation documented in the open source library \texttt{libpointmatcher}.
}
\label{tab:icpConfigAppSmartter}
\centering
\footnotesize
\begin{tabularx}{\textwidth}{ll X}

\toprule
\emph{Step} & \emph{Module} & \emph{Description} \\
\midrule
DF\textsubscript{read}
  & \texttt{SurfaceNormal} & Compute normal with  20 \acx{nn} and an approximation factor $\epsilon=3.16$. \\
  & \texttt{MaxDensity} & Subsample to keep point with density of 5\,pts/m$^3$.\\
\midrule
DF\textsubscript{ref}
  & \texttt{MaxDist} & Keep points within a radius of 40\,m from the last sensor pose.\\
  & \texttt{SurfaceNormal} & Compute normal and density with  20 \acx{nn} and an approximation factor $\epsilon=3.16$. \\
  & \texttt{MaxDensity} & Subsample to keep point with density of 10\,pts/m$^3$.\\
  & \texttt{MaxPointCount} & Subsample 70\,\% if there is more than 900,000 points.\\
\midrule
MF
  & \texttt{KDTree} & Use a kD-tree with a maximum matching distance of 1.5\,m.  \\
\midrule
OF
  & \texttt{TrimmedDist} & Keep 70\,\% closest paired points. \\
\midrule
EM
  & \texttt{PointToPlane} & Objective function using point-to-point \mbox{error}.\\
\midrule
TC
  & \texttt{Differential} & Stop after a minimum error below 0.01\,m and 0.001\,rad.\\
  & \texttt{Counter} & Stop after the iteration count reached 100. \\
  & \texttt{Bound} & Stop if transformation increases beyond 5.0\,m and 0.8\,rad.\\
\bottomrule

\end{tabularx}
\raggedright
\scriptsize
\emph{Legend}:\\ DF\textsubscript{read} = Data Filters for readings, DF\textsubscript{ref} = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.
\end{table}

% Describe 3D results
The complete \icp solution ran at 3 Hz, which permits the registration to run in real-time with a bit of margin.
As showed in \autoref{fig:smartter_map1}, the SmartTer started and ended its path in a street situated at the top left corner of the aerial view.
During the drive, a total of four loops were realized: two loops were made counterclockwise around the \acx{eth} main building (\textit{Hauptgeb\"{a}ude}), and two other loops clockwise around the Hospital (\textit{Universit\"{a}tsspital Z\"{u}rich}).
Even with the low density of point used, a reasonable amount of details could be preserved, as depicted in \autoref{fig:smartter_map2}, where even wires powering trams are visible over the street junction at the bottom of the image.
At the first loop-closing point after 660\,m, \icp-based odometry had accumulated an alignment error of 4.5\,m on the z-axis.
At the second loop-closing point after 920\,m, the error on the z-axis was 14.3\,m.
In both cases, the error was recovered and registration could proceed further without any problem
It is interesting to note that, at those two closing points, errors on z-axis were the most predominant of the 6 \acx{dof} possible.
When looking at each scan separately, we observed that the ground area displaying a usable density of points had a radius of 20\,m around the car. 
Also, the distance between two scans was often 8\,m, going as high as 17\,m when accelerating.
Urban environment are often referred as canyon-shaped, which means that the sides are well constrained by the buildings, but lacking points on the ground can lead to drift on pitch angle or/and on z-axis.  
The fast motion of the car coupled with the short range measurements gave little overlap to stabilize the elevation or the pitch angle and the error cumulated during each loop.
When the car was passing again in streets previously explored, the global map was reused successfully by recovering the large offset at loop closure points.
Although the 3D map mainly follows the aerial map, the combined impact of scanning rate and velocity of the platform brings the utility of laser odometry to its limit.
This application results are the largest presented in this section, with a total path length of 3.8\,km.

%We thank Rudolph Triebel and Luciano Spinello (\acx{asl}) for recording and preparing the scans presented in this section.

\begin{figure}[htbp]
\centering
	\includegraphics[width=0.79\textwidth]{smartter_top_overlaid_large_text}
\caption[Overlay of a large scale reconstruction of the \acx{eth} main building and its surroundings with an aerial view.]
{Overlay of a large scale reconstruction of the \acx{eth} main building and its surroundings with an aerial view.
Colors represent the elevation, dark blue being low and dark red being high.
Loop closing points are marked by numbers in circle, (1) being the small loop, (2) being the larger loop.
The orthogonal projection of the aerial image was provided by the \textit{Bundesamt f\"{u}r Landestopografie} swisstopo (Art. 30 GeoIV): 5704 000 000
}
\label{fig:smartter_map1}
\end{figure}

\begin{figure}[htbp]
\centering
	\includegraphics[width=0.79\textwidth]{smartter_side_large_text}
\caption[Bird's-eye view of the \acx{eth} main building.]
{
Bird's-eye view of the \acx{eth} main building.
Colors represent elevation, light gray being low, dark blue being high.
}
\label{fig:smartter_map2}
\end{figure}